In [2]:
# import pandas as pd
# from scipy import stats

# df1 = pd.read_csv('investment_2023-12-11.csv')
# df2 = pd.read_csv('investment_S2_2023-12-13.csv')


# # Merge datasets on 'SubjectID' to compare the same participants
# # Merge datasets on 'SubjectID' to compare the same participants
# merged_df = pd.merge(df1, df2, on='SubjectID', suffixes=('_day1', '_day2'))

# # Ensure both 'Parts Completed' columns for both days are not NaN by using inner join
# merged_df = merged_df.dropna(subset=['Parts Completed_day1', 'Parts Completed_day2'])

# # Now, the lengths of parts_completed_day1 and parts_completed_day2 should be equal
# parts_completed_day1 = merged_df['Parts Completed_day1']
# parts_completed_day2 = merged_df['Parts Completed_day2']

# # Perform the paired t-test
# t_stat, p_value = stats.ttest_rel(parts_completed_day1, parts_completed_day2)

# print(f'T-statistic: {t_stat}, P-value: {p_value}')

T-statistic: 82.4806445914858, P-value: 0.0


In [3]:
# from statsmodels.stats.power import TTestIndPower

# # Parameters for power analysis
# effect_size = 0.5  # assumed effect size (small=0.2, medium=0.5, large=0.8)
# alpha = 0.05  # significance level
# power = 0.8  # desired power

# analysis = TTestIndPower()
# sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1, alternative='two-sided')

# print(f'Sample Size: {sample_size}')

Sample Size: 63.765611775409695
